Tests to implement:

1. KDD/GDD biases
2. LH/SH biases
3. Do these biases scale with ag production
4. Do these biases change over the growing season
5. Do historical cmip6 temperature biases predict future changes

In [22]:
import xarray as xr
import xesmf as xe
import numpy as np
import matplotlib.pyplot as plt
import scipy
import statsmodels.api as sm
import cartopy
import cartopy.util
import cartopy.crs as ccrs
import glob
import sys, os
import pickle, gzip
import datetime

In [23]:
run ../util/setupConsole_su

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
dirCmip6 = '/home/edcoffel/drive/MAX-Filer/Research/Climate-02/Data-02-edcoffel-F20/CMIP6'
dirERA5 = '/home/edcoffel/drive/MAX-Filer/Research/Climate-02/Data-02-edcoffel-F20/ERA5'
dirDeepak = '/home/edcoffel/drive/MAX-Filer/Research/Climate-01/Personal-F20/edcoffel-F20/data/projects/ag-land-climate/deepak'
dirAgData = '/home/edcoffel/drive/MAX-Filer/Research/Climate-01/Personal-F20/edcoffel-F20/data/projects/ag-land-climate'
dirProj = '/home/edcoffel/drive/MAX-Filer/Research/Climate-01/Personal-F20/edcoffel-F20/research/2020-ag-cmip6'

In [25]:
# files = glob.glob('%s/cmip6_output/*.nc'%dirProj)
# for f in files:
#     os.rename(f,'%s.nc'%f[0:-4]) 

In [26]:
with gzip.open('%s/gdd-kdd-lat-era5.dat'%dirAgData, 'rb') as f:
    era5_lat = pickle.load(f)
with gzip.open('%s/gdd-kdd-lon-era5.dat'%dirAgData, 'rb') as f:
    era5_lon = pickle.load(f)

In [27]:
cmip6_models = ['access-cm2', 'access-esm1-5', 'awi-cm-1-1-mr', 'bcc-csm2-mr', 'bcc-esm1', 'canesm5', 'ec-earth3', \
                'gfdl-cm4', 'gfdl-esm4', 'giss-e2-1-g', 'kace-1-0-g', 'fgoals-g3', 'inm-cm5-0', 'ipsl-cm6a-lr', 'miroc6', \
                'mpi-esm1-2-hr', 'mpi-esm1-2-lr', 'mri-esm2-0', 'noresm2-lm', 'noresm2-mm', 'sam0-unicon']

In [28]:
region = 'global'

if region == 'global':
    latRange = [-90, 90]
    lonRange = [0, 360]
elif region == 'us':
    latRange = [20, 57]
    lonRange = [220, 300]
elif region == 'china':
    latRange = [25, 45]
    lonRange = [105, 123]
elif region == 'eu':
    latRange = [38, 56]
    lonRange = [-5, 40]

latIndRange_era5 = np.where((era5_lat >= latRange[0]) & (era5_lat <= latRange[1]))[0]
lonIndRange_era5 = np.where((era5_lon >= lonRange[0]) & (era5_lon <= lonRange[1]))[0]

regridMesh_current = xr.Dataset({'lat': (['lat'], np.arange(latRange[0], latRange[1], 1.5)),
                            'lon': (['lon'], np.arange(lonRange[0], lonRange[1], 1.5)),})

regridMesh_global = xr.Dataset({'lat': (['lat'], np.arange(-90, 90, 1.5)),
                                'lon': (['lon'], np.arange(0, 360, 1.5)),})


if region == 'global':
    regridMesh = regridMesh_global
else:
    regridMesh = regridMesh_current

In [29]:
rebuild = False

if rebuild:
    for m, model in enumerate(cmip6_models):
        print('loading %s'%model)

        if os.path.isfile('cmip6_output/cmip6_tasmax_max_%s_regrid_%s.nc'%(region, cmip6_models[m])):
            print('skipping %s, exists'%cmip6_models[m])
            continue

        if not os.path.isfile('cmip6_output/cmip6_tasmax_max_global_%s.nc'%(cmip6_models[m])):
            print('skipping %s, base file DOESNT EXIST'%cmip6_models[m])
            continue

        ds_global_txx = xr.open_dataset('cmip6_output/cmip6_tasmax_max_global_%s.nc'%cmip6_models[m])
        ds_global_monthly_txx = xr.open_dataset('cmip6_output/cmip6_tasmax_monthly_max_global_%s.nc'%cmip6_models[m])
        ds_global_t50p = xr.open_dataset('cmip6_output/cmip6_tasmax_mean_global_%s.nc'%cmip6_models[m])
        
        # add cyclic point before regridding
        lon_data = cartopy.util.add_cyclic_point(ds_global_txx.lon)
        temp_data = cartopy.util.add_cyclic_point(ds_global_txx.tasmax_max)

        da_global_txx_cyc = xr.DataArray(data   = temp_data, 
                          dims   = ['time', 'lat', 'lon'],
                          coords = {'time':ds_global_txx.time, 'lat':ds_global_txx.lat, 'lon':lon_data},
                          attrs  = {'units'     : 'C'
                            })
        ds_global_txx_cyc = xr.Dataset()
        ds_global_txx_cyc['tasmax_max'] = da_global_txx_cyc
        
        regridder = xe.Regridder(ds_global_txx_cyc, regridMesh, 'bilinear', reuse_weights=True)
        regridder.clean_weight_file()
        ds_global_txx = regridder(ds_global_txx_cyc)

        
        # add cyclic point before regridding
        lon_data = cartopy.util.add_cyclic_point(ds_global_monthly_txx.lon)
        temp_data = cartopy.util.add_cyclic_point(ds_global_monthly_txx.tasmax_monthly_max)

        da_global_monthly_txx_cyc = xr.DataArray(data   = temp_data, 
                          dims   = ['time', 'lat', 'lon'],
                          coords = {'time':ds_global_monthly_txx.time, 'lat':ds_global_monthly_txx.lat, 'lon':lon_data},
                          attrs  = {'units'     : 'C'
                            })
        ds_global_monthly_txx_cyc = xr.Dataset()
        ds_global_monthly_txx_cyc['tasmax_monthly_max'] = da_global_monthly_txx_cyc
        
        regridder = xe.Regridder(ds_global_monthly_txx_cyc, regridMesh, 'bilinear', reuse_weights=True)
        regridder.clean_weight_file()
        ds_global_monthly_txx = regridder(ds_global_monthly_txx_cyc)

        
        # add cyclic point before regridding
        lon_data = cartopy.util.add_cyclic_point(ds_global_t50p.lon)
        temp_data = cartopy.util.add_cyclic_point(ds_global_t50p.tasmax_mean)

        da_global_t50p_cyc = xr.DataArray(data   = temp_data, 
                          dims   = ['time', 'lat', 'lon'],
                          coords = {'time':ds_global_t50p.time, 'lat':ds_global_t50p.lat, 'lon':lon_data},
                          attrs  = {'units'     : 'C'
                            })
        ds_global_t50p_cyc = xr.Dataset()
        ds_global_t50p_cyc['tasmax_mean'] = da_global_t50p_cyc
        
        regridder = xe.Regridder(ds_global_t50p_cyc, regridMesh, 'bilinear', reuse_weights=True)
        regridder.clean_weight_file()
        ds_global_t50p = regridder(ds_global_t50p_cyc)
        
        cmip6_tasmax_max = ds_global_txx.assign_coords({'model':model})
        cmip6_tasmax_monthly_max = ds_global_monthly_txx.assign_coords({'model':model})
        cmip6_tasmax_mean = ds_global_t50p.assign_coords({'model':model})

        cmip6_tasmax_max.to_netcdf('cmip6_output/cmip6_tasmax_max_regrid_%s_%s.nc'%(region, model))
        cmip6_tasmax_monthly_max.to_netcdf('cmip6_output/cmip6_tasmax_monthly_max_regrid_%s_%s.nc'%(region, model))
        cmip6_tasmax_mean.to_netcdf('cmip6_output/cmip6_tasmax_mean_regrid_%s_%s.nc'%(region, model))


In [30]:
crop = 'Maize'
rebuild = False

if rebuild:
    for m, model in enumerate(cmip6_models):
        print('loading %s'%model)

#         if os.path.isfile('cmip6_output/growing_season/cmip6_%s_tasmax_grow_max_%s_%s_regrid.nc'%(crop, region, model)):
#             print('skipping %s, exists'%cmip6_models[m])
#             continue

        if not os.path.isfile('cmip6_output/growing_season/cmip6_Maize_grow_tasmax_max_global_%s.nc'%model):
            print('skipping %s, base file DOESNT EXIST'%model)
            continue

        ds_global_tmax = xr.open_dataset('cmip6_output/growing_season/cmip6_Maize_grow_tasmax_max_global_%s.nc'%model)
        ds_global_tmean = xr.open_dataset('cmip6_output/growing_season/cmip6_Maize_grow_tasmax_mean_global_%s.nc'%model)
        
        # add cyclic point before regridding
        lon_data = cartopy.util.add_cyclic_point(ds_global_tmax.lon)
        temp_data = cartopy.util.add_cyclic_point(ds_global_tmax.tasmax_grow_max)

        da_global_tasmax_grow_tmax_cyc = xr.DataArray(data   = temp_data, 
                          dims   = ['time', 'lat', 'lon'],
                          coords = {'time':ds_global_tmax.time, 'lat':ds_global_tmax.lat, 'lon':lon_data},
                          attrs  = {'units'     : 'C'
                            })
        
        ds_global_tasmax_grow_tmax_cyc = xr.Dataset()
        ds_global_tasmax_grow_tmax_cyc['tasmax_grow_max'] = da_global_tasmax_grow_tmax_cyc
        
        regridder = xe.Regridder(ds_global_tasmax_grow_tmax_cyc, regridMesh, 'bilinear', reuse_weights=True)
        regridder.clean_weight_file()
        ds_global_tasmax_grow_tmax_cyc_regrid = regridder(ds_global_tasmax_grow_tmax_cyc)
        
        
        
        # add cyclic point before regridding
        lon_data = cartopy.util.add_cyclic_point(ds_global_tmean.lon)
        temp_data = cartopy.util.add_cyclic_point(ds_global_tmean.tasmax_grow_mean)

        da_global_tasmax_grow_tmean_cyc = xr.DataArray(data   = temp_data, 
                          dims   = ['time', 'lat', 'lon'],
                          coords = {'time':ds_global_tmean.time, 'lat':ds_global_tmean.lat, 'lon':lon_data},
                          attrs  = {'units'     : 'C'
                            })
        
        ds_global_tasmax_grow_tmean_cyc = xr.Dataset()
        ds_global_tasmax_grow_tmean_cyc['tasmax_grow_mean'] = da_global_tasmax_grow_tmean_cyc
        
        regridder = xe.Regridder(ds_global_tasmax_grow_tmean_cyc, regridMesh, 'bilinear', reuse_weights=True)
        regridder.clean_weight_file()
        ds_global_tasmax_grow_tmean_cyc_regrid = regridder(ds_global_tasmax_grow_tmean_cyc)

        ds_global_tasmax_grow_tmax_cyc_regrid = ds_global_tasmax_grow_tmax_cyc_regrid.assign_coords({'model':model})
        ds_global_tasmax_grow_tmean_cyc_regrid = ds_global_tasmax_grow_tmean_cyc_regrid.assign_coords({'model':model})

        ds_global_tasmax_grow_tmax_cyc_regrid.to_netcdf('cmip6_output/growing_season/cmip6_%s_tasmax_grow_max_%s_%s_regrid.nc'%(crop, region, model))
        ds_global_tasmax_grow_tmean_cyc_regrid.to_netcdf('cmip6_output/growing_season/cmip6_%s_tasmax_grow_mean_%s_%s_regrid.nc'%(crop, region, model))
        
        if os.path.isfile('cmip6_output/growing_season/cmip6_%s_grow_ef_mon_global_%s.nc'%(crop, model)):
            print('loading ef for %s'%model)
            ds_global_ef = xr.open_dataset('cmip6_output/growing_season/cmip6_%s_grow_ef_mon_global_%s.nc'%(crop, model))
            
            # add cyclic point before regridding
            lon_data = cartopy.util.add_cyclic_point(ds_global_ef.lon)
            ef_data = cartopy.util.add_cyclic_point(ds_global_ef.grow_ef)

            da_global_ef_grow_cyc = xr.DataArray(data   = ef_data, 
                              dims   = ['time', 'lat', 'lon'],
                              coords = {'time':ds_global_ef.time, 'lat':ds_global_ef.lat, 'lon':lon_data},
                              attrs  = {'units'     : 'Fraction'
                                })

            ds_global_ef_grow_cyc = xr.Dataset()
            ds_global_ef_grow_cyc['grow_ef'] = da_global_ef_grow_cyc

            regridder = xe.Regridder(ds_global_ef_grow_cyc, regridMesh, 'bilinear', reuse_weights=True)
            regridder.clean_weight_file()
            ds_global_ef_grow_cyc_regrid = regridder(ds_global_ef_grow_cyc)
            
            ds_global_ef_grow_cyc_regrid = ds_global_ef_grow_cyc_regrid.assign_coords({'model':model})
            ds_global_ef_grow_cyc_regrid.to_netcdf('cmip6_output/growing_season/cmip6_%s_grow_ef_mon_global_%s_regrid.nc'%(crop, model))
            
        else:
            print('skipping ef %s, doesn\'t exists'%cmip6_models[m])
        


In [31]:
cmip6_tasmax_grow_max = xr.Dataset()
cmip6_tasmax_grow_mean = xr.Dataset()
cmip6_ef_grow = xr.Dataset()

ef_m = 0

for m, model in enumerate(cmip6_models):

    if not os.path.isfile('cmip6_output/growing_season/cmip6_%s_tasmax_grow_max_%s_%s_regrid.nc'%(crop, region, model)):
        print('skipping tasmax max %s'%model)
        
        continue
    
#     print('loading regridded growing season tasmax for %s'%model)

    ds_global_txx = xr.open_dataset('cmip6_output/growing_season/cmip6_%s_tasmax_grow_max_%s_%s_regrid.nc'%(crop, region, model))
    ds_global_t50p = xr.open_dataset('cmip6_output/growing_season/cmip6_%s_tasmax_grow_mean_%s_%s_regrid.nc'%(crop, region, model))
    
    if os.path.isfile('cmip6_output/growing_season/cmip6_%s_grow_ef_mon_global_%s_regrid.nc'%(crop, model)):
        ds_global_ef = xr.open_dataset('cmip6_output/growing_season/cmip6_%s_grow_ef_mon_global_%s_regrid.nc'%(crop, model))
        
        if ef_m == 0:
            cmip6_ef_grow = ds_global_ef
        else:
            cmip6_ef_grow = xr.concat([cmip6_ef_grow, ds_global_ef], dim='model')
        ef_m += 1
    else:
        print('skipping ef %s'%model)

    if m == 0:
        cmip6_tasmax_grow_max = ds_global_txx
        cmip6_tasmax_grow_mean = ds_global_t50p
    else:
        cmip6_tasmax_grow_max = xr.concat([cmip6_tasmax_grow_max, ds_global_txx], dim='model')
        cmip6_tasmax_grow_mean = xr.concat([cmip6_tasmax_grow_mean, ds_global_t50p], dim='model')


skipping ef gfdl-cm4
skipping ef gfdl-esm4
skipping ef kace-1-0-g
skipping ef sam0-unicon


In [32]:
cmip6_tasmax_max = xr.Dataset()
cmip6_tasmax_monthly_max = xr.Dataset()
cmip6_tasmax_mean = xr.Dataset()

for m, model in enumerate(cmip6_models):

    print('loading regridded tasmax for %s'%model)

    ds_global_txx = xr.open_dataset('cmip6_output/cmip6_tasmax_max_regrid_%s_%s.nc'%(region, model))
    ds_global_monthly_tx = xr.open_dataset('cmip6_output/cmip6_tasmax_monthly_max_regrid_%s_%s.nc'%(region, model))
    ds_global_t50p = xr.open_dataset('cmip6_output/cmip6_tasmax_mean_regrid_%s_%s.nc'%(region, model))

    if m == 0:
        cmip6_tasmax_max = ds_global_txx
        cmip6_tasmax_monthly_max = ds_global_monthly_tx
        cmip6_tasmax_mean = ds_global_t50p
    else:
        cmip6_tasmax_max = xr.concat([cmip6_tasmax_max, ds_global_txx], dim='model')
        cmip6_tasmax_monthly_max = xr.concat([cmip6_tasmax_monthly_max, ds_global_monthly_tx], dim='model')
        cmip6_tasmax_mean = xr.concat([cmip6_tasmax_mean, ds_global_t50p], dim='model')


loading regridded tasmax for access-cm2
loading regridded tasmax for access-esm1-5
loading regridded tasmax for awi-cm-1-1-mr
loading regridded tasmax for bcc-csm2-mr
loading regridded tasmax for bcc-esm1
loading regridded tasmax for canesm5
loading regridded tasmax for ec-earth3
loading regridded tasmax for gfdl-cm4
loading regridded tasmax for gfdl-esm4
loading regridded tasmax for giss-e2-1-g
loading regridded tasmax for kace-1-0-g
loading regridded tasmax for fgoals-g3
loading regridded tasmax for inm-cm5-0
loading regridded tasmax for ipsl-cm6a-lr
loading regridded tasmax for miroc6
loading regridded tasmax for mpi-esm1-2-hr
loading regridded tasmax for mpi-esm1-2-lr
loading regridded tasmax for mri-esm2-0
loading regridded tasmax for noresm2-lm
loading regridded tasmax for noresm2-mm
loading regridded tasmax for sam0-unicon


In [33]:
rebuild = False


if rebuild:
    for m, model in enumerate(cmip6_models):
        print('loading %s'%model)

        if os.path.isfile('cmip6_output/cmip6_tasmin_max_%s_regrid_%s.nc'%(region, model)):
            print('skipping %s, exists'%cmip6_models[m])
            continue

        if not os.path.isfile('cmip6_output/cmip6_tasmin_max_global_%s.nc'%(model)):
            print('skipping %s, base file DOESNT EXIST'%cmip6_models[m])
            continue

        ds_global_txx = xr.open_dataset('cmip6_output/cmip6_tasmin_max_global_%s.nc'%model)
        ds_global_monthly_txx = xr.open_dataset('cmip6_output/cmip6_tasmin_monthly_max_global_%s.nc'%model)
        ds_global_t50p = xr.open_dataset('cmip6_output/cmip6_tasmin_mean_global_%s.nc'%model)

        # add cyclic point before regridding
        lon_data = cartopy.util.add_cyclic_point(ds_global_txx.lon)
        temp_data = cartopy.util.add_cyclic_point(ds_global_txx.tasmin_max)

        da_global_txx_cyc = xr.DataArray(data   = temp_data, 
                          dims   = ['time', 'lat', 'lon'],
                          coords = {'time':ds_global_txx.time, 'lat':ds_global_txx.lat, 'lon':lon_data},
                          attrs  = {'units'     : 'C'
                            })
        ds_global_txx_cyc = xr.Dataset()
        ds_global_txx_cyc['tasmin_max'] = da_global_txx_cyc
        
        regridder = xe.Regridder(ds_global_txx_cyc, regridMesh, 'bilinear', reuse_weights=True)
        regridder.clean_weight_file()
        ds_global_txx = regridder(ds_global_txx_cyc)

        
        # add cyclic point before regridding
        lon_data = cartopy.util.add_cyclic_point(ds_global_monthly_txx.lon)
        temp_data = cartopy.util.add_cyclic_point(ds_global_monthly_txx.tasmin_monthly_max)

        da_global_monthly_txx_cyc = xr.DataArray(data   = temp_data, 
                          dims   = ['time', 'lat', 'lon'],
                          coords = {'time':ds_global_monthly_txx.time, 'lat':ds_global_monthly_txx.lat, 'lon':lon_data},
                          attrs  = {'units'     : 'C'
                            })
        ds_global_monthly_txx_cyc = xr.Dataset()
        ds_global_monthly_txx_cyc['tasmin_monthly_max'] = da_global_monthly_txx_cyc
        
        regridder = xe.Regridder(ds_global_monthly_txx_cyc, regridMesh, 'bilinear', reuse_weights=True)
        regridder.clean_weight_file()
        ds_global_monthly_txx = regridder(ds_global_monthly_txx_cyc)

        
        # add cyclic point before regridding
        lon_data = cartopy.util.add_cyclic_point(ds_global_t50p.lon)
        temp_data = cartopy.util.add_cyclic_point(ds_global_t50p.tasmin_mean)

        da_global_t50p_cyc = xr.DataArray(data   = temp_data, 
                          dims   = ['time', 'lat', 'lon'],
                          coords = {'time':ds_global_t50p.time, 'lat':ds_global_t50p.lat, 'lon':lon_data},
                          attrs  = {'units'     : 'C'
                            })
        ds_global_t50p_cyc = xr.Dataset()
        ds_global_t50p_cyc['tasmin_mean'] = da_global_t50p_cyc
        
        regridder = xe.Regridder(ds_global_t50p_cyc, regridMesh, 'bilinear', reuse_weights=True)
        regridder.clean_weight_file()
        ds_global_t50p = regridder(ds_global_t50p_cyc)

        cmip6_tasmin_max = ds_global_txx.assign_coords({'model':model})
        cmip6_tasmin_monthly_max = ds_global_monthly_txx.assign_coords({'model':model})
        cmip6_tasmin_mean = ds_global_t50p.assign_coords({'model':model})

        cmip6_tasmin_max.to_netcdf('cmip6_output/cmip6_tasmin_max_regrid_%s_%s.nc'%(region, model))
        cmip6_tasmin_monthly_max.to_netcdf('cmip6_output/cmip6_tasmin_monthly_max_regrid_%s_%s.nc'%(region, model))
        cmip6_tasmin_mean.to_netcdf('cmip6_output/cmip6_tasmin_mean_regrid_%s_%s.nc'%(region, model))


In [34]:
cmip6_tasmin_max = xr.Dataset()
cmip6_tasmin_monthly_max = xr.Dataset()
cmip6_tasmin_mean = xr.Dataset()

for m, model in enumerate(cmip6_models):
    
    if not os.path.isfile('cmip6_output/cmip6_tasmin_max_regrid_%s_%s.nc'%(region, model)):
        print('skipping %s, base file DOESNT EXIST'%model)
        continue
    
    print('loading regridded tasmin for %s'%model)

    ds_global_txx = xr.open_dataset('cmip6_output/cmip6_tasmin_max_regrid_%s_%s.nc'%(region, model))
    ds_global_monthly_tx = xr.open_dataset('cmip6_output/cmip6_tasmin_monthly_max_regrid_%s_%s.nc'%(region, model))
    ds_global_t50p = xr.open_dataset('cmip6_output/cmip6_tasmin_mean_regrid_%s_%s.nc'%(region, model))

    if m == 0:
        cmip6_tasmin_max = ds_global_txx
        cmip6_tasmin_monthly_max = ds_global_monthly_tx
        cmip6_tasmin_mean = ds_global_t50p
    else:
        cmip6_tasmin_max = xr.concat([cmip6_tasmin_max, ds_global_txx], dim='model')
        cmip6_tasmin_monthly_max = xr.concat([cmip6_tasmin_monthly_max, ds_global_monthly_tx], dim='model')
        cmip6_tasmin_mean = xr.concat([cmip6_tasmin_mean, ds_global_t50p], dim='model')


loading regridded tasmin for access-cm2
loading regridded tasmin for access-esm1-5
loading regridded tasmin for awi-cm-1-1-mr
skipping bcc-csm2-mr, base file DOESNT EXIST
loading regridded tasmin for bcc-esm1
loading regridded tasmin for canesm5
loading regridded tasmin for ec-earth3
skipping gfdl-cm4, base file DOESNT EXIST
skipping gfdl-esm4, base file DOESNT EXIST
loading regridded tasmin for giss-e2-1-g
skipping kace-1-0-g, base file DOESNT EXIST
loading regridded tasmin for fgoals-g3
loading regridded tasmin for inm-cm5-0
loading regridded tasmin for ipsl-cm6a-lr
loading regridded tasmin for miroc6
loading regridded tasmin for mpi-esm1-2-hr
loading regridded tasmin for mpi-esm1-2-lr
loading regridded tasmin for mri-esm2-0
loading regridded tasmin for noresm2-lm
loading regridded tasmin for noresm2-mm
skipping sam0-unicon, base file DOESNT EXIST


In [35]:
cmip6_tasmax_max_trend = xr.Dataset()
cmip6_tasmax_monthly_max_trend = xr.Dataset()
cmip6_tasmax_mean_trend = xr.Dataset()

for m, model in enumerate(cmip6_models):

    if not os.path.isfile('cmip6_output/cmip6_tasmax_max_trend_regrid_%s_%s.nc'%(region, model)):
        print('skipping %s, base file DOESNT EXIST'%model)
        continue
    
    print('loading trends for %s'%model)

    ds_global_txx_trend = xr.open_dataset('cmip6_output/cmip6_tasmax_max_trend_regrid_%s_%s.nc'%(region, model))
    ds_global_monthly_tx_trend = xr.open_dataset('cmip6_output/cmip6_tasmax_monthly_max_trend_regrid_%s_%s.nc'%(region, model))
    ds_global_t50p_trend = xr.open_dataset('cmip6_output/cmip6_tasmax_mean_trend_regrid_%s_%s.nc'%(region, model))

    if m == 0:
        cmip6_tasmax_max_trend = ds_global_txx_trend
        cmip6_tasmax_monthly_max_trend = ds_global_monthly_tx_trend
        cmip6_tasmax_mean_trend = ds_global_t50p_trend
    else:
        cmip6_tasmax_max_trend = xr.concat([cmip6_tasmax_max_trend, ds_global_txx_trend], dim='model')
        cmip6_tasmax_monthly_max_trend = xr.concat([cmip6_tasmax_monthly_max_trend, ds_global_monthly_tx_trend], dim='model')
        cmip6_tasmax_mean_trend = xr.concat([cmip6_tasmax_mean_trend, ds_global_t50p_trend], dim='model')

# cmip6_tasmax_max_trend = cmip6_tasmax_max_trend.assign_coords({'model':cmip6_models})
# cmip6_tasmax_monthly_max_trend = cmip6_tasmax_monthly_max_trend.assign_coords({'model':cmip6_models})
# cmip6_tasmax_mean_trend = cmip6_tasmax_mean_trend.assign_coords({'model':cmip6_models})


loading trends for access-cm2
loading trends for access-esm1-5
loading trends for awi-cm-1-1-mr
loading trends for bcc-csm2-mr
loading trends for bcc-esm1
loading trends for canesm5
loading trends for ec-earth3
loading trends for gfdl-cm4
loading trends for gfdl-esm4
loading trends for giss-e2-1-g
loading trends for kace-1-0-g
loading trends for fgoals-g3
loading trends for inm-cm5-0
loading trends for ipsl-cm6a-lr
loading trends for miroc6
loading trends for mpi-esm1-2-hr
loading trends for mpi-esm1-2-lr
loading trends for mri-esm2-0
loading trends for noresm2-lm
loading trends for noresm2-mm
loading trends for sam0-unicon


In [36]:
cmip6_tasmax_grow_max_trend = xr.Dataset()
cmip6_tasmax_grow_mean_trend = xr.Dataset()
cmip6_ef_grow_trend = xr.Dataset()

ef_m = 0

for m, model in enumerate(cmip6_models):

    if not os.path.isfile('cmip6_output/growing_season/cmip6_%s_%s_grow_max_trend_%s_%s_regrid.nc'%(crop, 'tasmax', region, model)):
        print('skipping %s, base file DOESNT EXIST'%model)
        continue
    
    print('loading growing season trends for %s'%model)

    if os.path.isfile('cmip6_output/growing_season/cmip6_%s_grow_ef_mon_trend_global_%s_regrid.nc'%(crop, model)):
        ds_global_ef_trend = xr.open_dataset('cmip6_output/growing_season/cmip6_%s_grow_ef_mon_trend_global_%s_regrid.nc'%(crop, model))
        
        if ef_m == 0:
            cmip6_ef_grow_trend = ds_global_ef_trend
        else:
            cmip6_ef_grow_trend = xr.concat([cmip6_ef_grow_trend, ds_global_ef_trend], dim='model')
        ef_m += 1
    else:
        print('skipping ef %s'%model)
    
    ds_global_txx_trend = xr.open_dataset('cmip6_output/growing_season/cmip6_%s_%s_grow_max_trend_%s_%s_regrid.nc'%(crop, 'tasmax', region, model))
    ds_global_t50p_trend = xr.open_dataset('cmip6_output/growing_season/cmip6_%s_%s_grow_mean_trend_%s_%s_regrid.nc'%(crop, 'tasmax', region, model))

    if m == 0:
        cmip6_tasmax_grow_max_trend = ds_global_txx_trend
        cmip6_tasmax_grow_mean_trend = ds_global_t50p_trend
    else:
        cmip6_tasmax_grow_max_trend = xr.concat([cmip6_tasmax_grow_max_trend, ds_global_txx_trend], dim='model')
        cmip6_tasmax_grow_mean_trend = xr.concat([cmip6_tasmax_grow_mean_trend, ds_global_t50p_trend], dim='model')

# cmip6_tasmax_max_trend = cmip6_tasmax_max_trend.assign_coords({'model':cmip6_models})
# cmip6_tasmax_monthly_max_trend = cmip6_tasmax_monthly_max_trend.assign_coords({'model':cmip6_models})
# cmip6_tasmax_mean_trend = cmip6_tasmax_mean_trend.assign_coords({'model':cmip6_models})


loading growing season trends for access-cm2
loading growing season trends for access-esm1-5
loading growing season trends for awi-cm-1-1-mr
loading growing season trends for bcc-csm2-mr
loading growing season trends for bcc-esm1
loading growing season trends for canesm5
loading growing season trends for ec-earth3
loading growing season trends for gfdl-cm4
skipping ef gfdl-cm4
loading growing season trends for gfdl-esm4
skipping ef gfdl-esm4
loading growing season trends for giss-e2-1-g
loading growing season trends for kace-1-0-g
skipping ef kace-1-0-g
loading growing season trends for fgoals-g3
loading growing season trends for inm-cm5-0
loading growing season trends for ipsl-cm6a-lr
loading growing season trends for miroc6
loading growing season trends for mpi-esm1-2-hr
loading growing season trends for mpi-esm1-2-lr
loading growing season trends for mri-esm2-0
loading growing season trends for noresm2-lm
loading growing season trends for noresm2-mm
loading growing season trends for

In [37]:
cmip6_tasmin_max_trend = xr.Dataset()
cmip6_tasmin_monthly_max_trend = xr.Dataset()
cmip6_tasmin_mean_trend = xr.Dataset()

for m, model in enumerate(cmip6_models):

    if not os.path.isfile('cmip6_output/cmip6_tasmin_max_trend_regrid_%s_%s.nc'%(region, model)):
        print('skipping %s, base file DOESNT EXIST'%model)
        continue
    
    print('loading trends for %s'%model)

    ds_global_txx_trend = xr.open_dataset('cmip6_output/cmip6_tasmin_max_trend_regrid_%s_%s.nc'%(region, model))
    ds_global_monthly_tx_trend = xr.open_dataset('cmip6_output/cmip6_tasmin_monthly_max_trend_regrid_%s_%s.nc'%(region, model))
    ds_global_t50p_trend = xr.open_dataset('cmip6_output/cmip6_tasmin_mean_trend_regrid_%s_%s.nc'%(region, model))

    if m == 0:
        cmip6_tasmin_max_trend = ds_global_txx_trend
        cmip6_tasmin_monthly_max_trend = ds_global_monthly_tx_trend
        cmip6_tasmin_mean_trend = ds_global_t50p_trend
    else:
        cmip6_tasmin_max_trend = xr.concat([cmip6_tasmin_max_trend, ds_global_txx_trend], dim='model')
        cmip6_tasmin_monthly_max_trend = xr.concat([cmip6_tasmin_monthly_max_trend, ds_global_monthly_tx_trend], dim='model')
        cmip6_tasmin_mean_trend = xr.concat([cmip6_tasmin_mean_trend, ds_global_t50p_trend], dim='model')

# cmip6_tasmin_max_trend = cmip6_tasmin_max_trend.assign_coords({'model':cmip6_models})
# cmip6_tasmin_monthly_max_trend = cmip6_tasmin_monthly_max_trend.assign_coords({'model':cmip6_models})
# cmip6_tasmin_mean_trend = cmip6_tasmin_mean_trend.assign_coords({'model':cmip6_models})


loading trends for access-cm2
loading trends for access-esm1-5
loading trends for awi-cm-1-1-mr
skipping bcc-csm2-mr, base file DOESNT EXIST
loading trends for bcc-esm1
loading trends for canesm5
loading trends for ec-earth3
skipping gfdl-cm4, base file DOESNT EXIST
skipping gfdl-esm4, base file DOESNT EXIST
loading trends for giss-e2-1-g
skipping kace-1-0-g, base file DOESNT EXIST
loading trends for fgoals-g3
loading trends for inm-cm5-0
loading trends for ipsl-cm6a-lr
loading trends for miroc6
loading trends for mpi-esm1-2-hr
loading trends for mpi-esm1-2-lr
loading trends for mri-esm2-0
loading trends for noresm2-lm
loading trends for noresm2-mm
skipping sam0-unicon, base file DOESNT EXIST


In [38]:
%%time

if os.path.isfile('era5/era5_tasmax_max_regrid_%s.nc'%region):
    print('loading pre-computed era5...')
    era5_tasmax_max_regrid = xr.open_dataset('era5/era5_tasmax_max_regrid_%s.nc'%region)
else:
    era5_tasmax_max = xr.open_dataset('era5/era5_tasmax_max_global.nc')
    era5_tasmax_max = era5_tasmax_max.sel(time=slice('1981', '2014'))

    regridder = xe.Regridder(era5_tasmax_max, regridMesh, 'bilinear', reuse_weights=True)
    regridder.clean_weight_file()
    era5_tasmax_max_regrid = regridder(era5_tasmax_max)
    era5_tasmax_max_regrid.to_netcdf('era5/era5_tasmax_max_regrid_%s.nc'%region)

if os.path.isfile('era5/era5_tasmax_monthly_max_regrid_%s.nc'%region):
    era5_tasmax_monthly_max_regrid = xr.open_dataset('era5/era5_tasmax_monthly_max_regrid_%s.nc'%region)
else:
    era5_tasmax_monthly_max = xr.open_dataset('era5/era5_tasmax_monthly_max_global.nc')
    era5_tasmax_monthly_max = era5_tasmax_monthly_max.sel(time=slice('1981', '2014'))

    regridder = xe.Regridder(era5_tasmax_monthly_max, regridMesh, 'bilinear', reuse_weights=True)
    regridder.clean_weight_file()
    era5_tasmax_monthly_max_regrid = regridder(era5_tasmax_monthly_max)
    era5_tasmax_monthly_max_regrid.to_netcdf('era5/era5_tasmax_monthly_max_regrid_%s.nc'%region)

if os.path.isfile('era5/era5_tasmax_mean_regrid_%s.nc'%region):
    era5_tasmax_mean_regrid = xr.open_dataset('era5/era5_tasmax_mean_regrid_%s.nc'%region)
else:
    era5_tasmax_mean = xr.open_dataset('era5/era5_tasmax_mean_global.nc')
    era5_tasmax_mean = era5_tasmax_mean.sel(time=slice('1981', '2014'))

    regridder = xe.Regridder(era5_tasmax_mean, regridMesh, 'bilinear', reuse_weights=True)
    regridder.clean_weight_file()
    era5_tasmax_mean_regrid = regridder(era5_tasmax_mean)
    era5_tasmax_mean_regrid.to_netcdf('era5/era5_tasmax_mean_regrid_%s.nc'%region)


loading pre-computed era5...
CPU times: user 16.2 ms, sys: 90 µs, total: 16.2 ms
Wall time: 15.6 ms


In [43]:
# load era5 growing season temps

rebuild = False

if rebuild or not os.path.isfile('era5/growing_season/era5_%s_tasmax_grow_max_regrid_%s.nc'%(crop,region)):

    era5_tasmax_grow_max = xr.Dataset()
    era5_tasmax_grow_mean = xr.Dataset()

    era5_ef_grow = xr.Dataset()
    
    for y, year in enumerate(range(1981, 2014+1)):

        print('loading era5 growing season for %d'%year)

        ds_global_txx = xr.open_dataset('era5/growing_season/era5_%s_tasmax_grow_max_global_%s.nc'%(crop, year))
        ds_global_t50p = xr.open_dataset('era5/growing_season/era5_%s_tasmax_grow_mean_global_%s.nc'%(crop, year))

        ds_global_ef = xr.open_dataset('era5/growing_season/era5_%s_ef_grow_global_%s.nc'%(crop, year))
        
        if y == 0:
            era5_tasmax_grow_max = ds_global_txx
            era5_tasmax_grow_mean = ds_global_t50p
            
            era5_ef_grow = ds_global_ef
        else:
            era5_tasmax_grow_max = xr.concat([era5_tasmax_grow_max, ds_global_txx], dim='time')
            era5_tasmax_grow_mean = xr.concat([era5_tasmax_grow_mean, ds_global_t50p], dim='time')
            
            era5_ef_grow = xr.concat([era5_ef_grow, ds_global_ef], dim='time')

    regridder = xe.Regridder(era5_tasmax_grow_max, regridMesh, 'bilinear', reuse_weights=True)
    regridder.clean_weight_file()
    era5_tasmax_grow_max_regrid = regridder(era5_tasmax_grow_max)
    era5_tasmax_grow_max_regrid.to_netcdf('era5/growing_season/era5_%s_tasmax_grow_max_regrid_%s.nc'%(crop,region))

    regridder = xe.Regridder(era5_tasmax_grow_mean, regridMesh, 'bilinear', reuse_weights=True)
    regridder.clean_weight_file()
    era5_tasmax_grow_mean_regrid = regridder(era5_tasmax_grow_mean)
    era5_tasmax_grow_mean_regrid.to_netcdf('era5/growing_season/era5_%s_tasmax_grow_mean_regrid_%s.nc'%(crop,region))
    
    regridder = xe.Regridder(era5_ef_grow, regridMesh, 'bilinear', reuse_weights=True)
    regridder.clean_weight_file()
    era5_ef_grow_regrid = regridder(era5_ef_grow)
    era5_ef_grow_regrid.to_netcdf('era5/growing_season/era5_%s_ef_grow_regrid_%s.nc'%(crop,region))
else:
    era5_tasmax_grow_max_regrid = xr.open_dataset('era5/growing_season/era5_%s_tasmax_grow_max_regrid_%s.nc'%(crop,region))
    era5_tasmax_grow_mean_regrid = xr.open_dataset('era5/growing_season/era5_%s_tasmax_grow_mean_regrid_%s.nc'%(crop,region))
    era5_ef_grow_regrid = xr.open_dataset('era5/growing_season/era5_%s_ef_grow_regrid_%s.nc'%(crop,region))
    


In [44]:
%%time

recalc = False

# era5 trends for growing season data

if recalc:

    # calc era5 txx and t50p trends
    era5_tasmax_grow_max_trend = xr.Dataset()
    era5_tasmax_grow_mean_trend = xr.Dataset()
    era5_ef_grow_trend = xr.Dataset()

    print('calc era5 txx, t50p, and ef trends')
    tmp_tasmax_grow_max_trend = np.full([era5_tasmax_grow_max_regrid.lat.values.shape[0], era5_tasmax_grow_max_regrid.lon.values.shape[0]], np.nan)
    tmp_tasmax_grow_mean_trend = np.full([era5_tasmax_grow_mean_regrid.lat.values.shape[0], era5_tasmax_grow_mean_regrid.lon.values.shape[0]], np.nan)
    tmp_ef_grow_trend = np.full([era5_ef_grow_regrid.lat.values.shape[0], era5_ef_grow_regrid.lon.values.shape[0]], np.nan)

    for xlat in range(tmp_tasmax_grow_max_trend.shape[0]):
        for ylon in range(tmp_tasmax_grow_max_trend.shape[1]):
            cur_tasmax_grow_max = era5_tasmax_grow_max_regrid.tasmax_grow_max.values[:, xlat, ylon]
            X = sm.add_constant(range(1981, 2014+1))
            mdl = sm.OLS(cur_tasmax_grow_max, X).fit()
            tmp_tasmax_grow_max_trend[xlat, ylon] = mdl.params[1]*10

            cur_tasmax_grow_mean = era5_tasmax_grow_mean_regrid.tasmax_grow_mean.values[:, xlat, ylon]
            X = sm.add_constant(range(1981, 2014+1))
            mdl = sm.OLS(cur_tasmax_grow_mean, X).fit()
            tmp_tasmax_grow_mean_trend[xlat, ylon] = mdl.params[1]*10
            
            cur_ef_grow = era5_ef_grow_regrid.ef_grow.values[:, xlat, ylon]
            X = sm.add_constant(range(1981, 2014+1))
            mdl = sm.OLS(cur_ef_grow, X).fit()
            tmp_ef_grow_trend[xlat, ylon] = mdl.params[1]*10

    era5_tasmax_grow_max_trend['tasmax_grow_max_trend'] = xr.DataArray(data   = tmp_tasmax_grow_max_trend, 
                      dims   = ['lat', 'lon'],
                      coords = {'lat':era5_tasmax_grow_max_regrid.lat, 'lon':era5_tasmax_grow_max_regrid.lon},
                      attrs  = {'units'     : 'C'
                        })
    era5_tasmax_grow_max_trend.to_netcdf('era5/growing_season/era5_tasmax_grow_max_trend_%s.nc'%region)

    era5_tasmax_grow_mean_trend['tasmax_grow_mean_trend'] = xr.DataArray(data   = tmp_tasmax_grow_mean_trend, 
                      dims   = ['lat', 'lon'],
                      coords = {'lat':era5_tasmax_grow_mean_regrid.lat, 'lon':era5_tasmax_grow_mean_regrid.lon},
                      attrs  = {'units'     : 'C'
                        })
    era5_tasmax_grow_mean_trend.to_netcdf('era5/growing_season/era5_tasmax_grow_mean_trend_%s.nc'%region)
    
    era5_ef_grow_trend['ef_grow_trend'] = xr.DataArray(data   = tmp_ef_grow_trend, 
                      dims   = ['lat', 'lon'],
                      coords = {'lat':era5_ef_grow_regrid.lat, 'lon':era5_ef_grow_regrid.lon},
                      attrs  = {'units'     : 'C'
                        })
    era5_ef_grow_trend.to_netcdf('era5/growing_season/era5_ef_grow_trend_%s.nc'%region)
else:
    era5_tasmax_grow_max_trend = xr.open_dataset('era5/growing_season/era5_tasmax_grow_max_trend_%s.nc'%region)
    era5_tasmax_grow_mean_trend = xr.open_dataset('era5/growing_season/era5_tasmax_grow_mean_trend_%s.nc'%region)    
    era5_ef_grow_trend = xr.open_dataset('era5/growing_season/era5_ef_grow_trend_%s.nc'%region)




CPU times: user 15.5 ms, sys: 0 ns, total: 15.5 ms
Wall time: 136 ms


In [45]:
%%time

# era5 trends over whole year

recalc = False

if recalc:
    # calc era5 txx and t50p trends
    era5_tasmax_max_trend = xr.Dataset()
    era5_tasmax_monthly_max_trend = xr.Dataset()
    era5_tasmax_mean_trend = xr.Dataset()

    era5_tasmin_max_trend = xr.Dataset()
    era5_tasmin_mean_trend = xr.Dataset()

    print('calc era5 txx and t50p trends')
    tmp_tasmax_max_trend = np.full([era5_tasmax_max_regrid.lat.values.shape[0], era5_tasmax_max_regrid.lon.values.shape[0]], np.nan)
    tmp_tasmax_monthly_max_trend = np.full([12, era5_tasmax_max_regrid.lat.values.shape[0], era5_tasmax_max_regrid.lon.values.shape[0]], np.nan)
    tmp_tasmax_mean_trend = np.full([era5_tasmax_mean_regrid.lat.values.shape[0], era5_tasmax_mean_regrid.lon.values.shape[0]], np.nan)

    # tmp_tasmin_max_trend = np.full([era5_tasmin_max_regrid.lat.values.shape[0], era5_tasmin_max_regrid.lon.values.shape[0]], np.nan)
    # tmp_tasmin_mean_trend = np.full([era5_tasmin_mean_regrid.lat.values.shape[0], era5_tasmin_mean_regrid.lon.values.shape[0]], np.nan)

    monthly_groups = era5_tasmax_monthly_max_regrid.groupby('time.month').groups

    for xlat in range(tmp_tasmax_max_trend.shape[0]):
        for ylon in range(tmp_tasmax_max_trend.shape[1]):
            cur_tasmax_max = era5_tasmax_max_regrid.tasmax_max.values[:, xlat, ylon]
            X = sm.add_constant(range(1981, 2014+1))
            mdl = sm.OLS(cur_tasmax_max, X).fit()
            tmp_tasmax_max_trend[xlat, ylon] = mdl.params[1]*10

            cur_tasmax_mean = era5_tasmax_mean_regrid.tasmax_mean.values[:, xlat, ylon]
            X = sm.add_constant(range(1981, 2014+1))
            mdl = sm.OLS(cur_tasmax_mean, X).fit()
            tmp_tasmax_mean_trend[xlat, ylon] = mdl.params[1]*10

    #         cur_tasmin_max = era5_tasmin_max_regrid.tasmin_max.values[:, xlat, ylon]
    #         X = sm.add_constant(range(1981, 2014+1))
    #         mdl = sm.OLS(cur_tasmin_max, X).fit()
    #         tmp_tasmin_max_trend[xlat, ylon] = mdl.params[1]*10

    #         cur_tasmin_mean = era5_tasmin_mean_regrid.tasmin_mean.values[:, xlat, ylon]
    #         X = sm.add_constant(range(1981, 2014+1))
    #         mdl = sm.OLS(cur_tasmin_mean, X).fit()
    #         tmp_tasmin_mean_trend[xlat, ylon] = mdl.params[1]*10

            for month in range(1, 13):
                curMonthlyTx = era5_tasmax_monthly_max_regrid['tasmax_monthly_max'].values[monthly_groups[month], xlat, ylon]
                X = sm.add_constant(range(1981, 2014+1))
                mdl = sm.OLS(curMonthlyTx, X).fit()
                tmp_tasmax_monthly_max_trend[month-1, xlat, ylon] = mdl.params[1]*10

    era5_tasmax_max_trend['tasmax_max_trend'] = xr.DataArray(data   = tmp_tasmax_max_trend, 
                      dims   = ['lat', 'lon'],
                      coords = {'lat':era5_tasmax_max_regrid.lat, 'lon':era5_tasmax_max_regrid.lon},
                      attrs  = {'units'     : 'C'
                        })
    era5_tasmax_max_trend.to_netcdf('era5/era5_tasmax_max_trend_%s.nc'%region)

    era5_tasmax_monthly_max_trend['tasmax_monthly_max_trend'] = xr.DataArray(data   = tmp_tasmax_monthly_max_trend, 
                      dims   = ['month', 'lat', 'lon'],
                      coords = {'month':np.arange(1,13), 'lat':era5_tasmax_monthly_max_regrid.lat, 'lon':era5_tasmax_monthly_max_regrid.lon},
                      attrs  = {'units'     : 'C'
                        })
    era5_tasmax_monthly_max_trend.to_netcdf('era5/era5_tasmax_monthly_max_trend_%s.nc'%region)

    era5_tasmax_mean_trend['tasmax_mean_trend'] = xr.DataArray(data   = tmp_tasmax_mean_trend, 
                      dims   = ['lat', 'lon'],
                      coords = {'lat':era5_tasmax_mean_regrid.lat, 'lon':era5_tasmax_mean_regrid.lon},
                      attrs  = {'units'     : 'C'
                        })
    era5_tasmax_mean_trend.to_netcdf('era5/era5_tasmax_mean_trend_%s.nc'%region)

    era5_tasmin_max_trend['tasmin_max_trend'] = xr.DataArray(data   = tmp_tasmin_max_trend, 
                      dims   = ['lat', 'lon'],
                      coords = {'lat':era5_tasmin_max_regrid.lat, 'lon':era5_tasmin_max_regrid.lon},
                      attrs  = {'units'     : 'C'
                        })

    era5_tasmin_mean_trend['tasmin_mean_trend'] = xr.DataArray(data   = tmp_tasmin_mean_trend, 
                      dims   = ['lat', 'lon'],
                      coords = {'lat':era5_tasmin_mean_regrid.lat, 'lon':era5_tasmin_mean_regrid.lon},
                      attrs  = {'units'     : 'C'
                        })
else:
    era5_tasmax_max_trend = xr.open_dataset('era5/era5_tasmax_max_trend_%s.nc'%region)
    era5_tasmax_monthly_max_trend = xr.open_dataset('era5/era5_tasmax_monthly_max_trend_%s.nc'%region)
    era5_tasmax_mean_trend = xr.open_dataset('era5/era5_tasmax_mean_trend_%s.nc'%region)



CPU times: user 11.1 ms, sys: 3.98 ms, total: 15.1 ms
Wall time: 82.5 ms


In [46]:

# calculate trend bias between cmip6 and era5

recalc = True

if recalc:
    # TASMAX ANNUAL MAX
    print('tasmax annual max')
    cmip6_era5_tasmax_max_trend_bias_sig = np.full(era5_tasmax_max_trend.tasmax_max_trend.shape, np.nan)
    cmip6_era5_tasmax_max_trend_bias = np.full([len(cmip6_models), era5_tasmax_max_trend.tasmax_max_trend.shape[0], era5_tasmax_max_trend.tasmax_max_trend.shape[1]], np.nan)

    for m, model in enumerate(cmip6_models):
        cmip6_era5_tasmax_max_trend_bias[m,:,:] = cmip6_tasmax_max_trend.tasmax_max_trend.values[m,:,:] - era5_tasmax_max_trend.tasmax_max_trend.values

    for xlat in range(cmip6_era5_tasmax_max_trend_bias_sig.shape[0]):
        for ylon in range(cmip6_era5_tasmax_max_trend_bias_sig.shape[1]):
            t, p = scipy.stats.ttest_1samp(cmip6_era5_tasmax_max_trend_bias[:, xlat, ylon], 0)
            cmip6_era5_tasmax_max_trend_bias_sig[xlat, ylon] = p

    # TASMAX MONTHLY MAX
    print('tasmax monthly max')
    cmip6_era5_tasmax_monthly_max_trend_bias_sig = np.full([12, era5_tasmax_monthly_max_trend.tasmax_monthly_max_trend.shape[1], era5_tasmax_monthly_max_trend.tasmax_monthly_max_trend.shape[2]], np.nan)
    cmip6_era5_tasmax_monthly_max_trend_bias = np.full([12, len(cmip6_models), era5_tasmax_monthly_max_trend.tasmax_monthly_max_trend.shape[1], era5_tasmax_monthly_max_trend.tasmax_monthly_max_trend.shape[2]], np.nan)

    for m, model in enumerate(cmip6_models):
        for month in range(12):
            cmip6_era5_tasmax_monthly_max_trend_bias[month,m,:,:] = cmip6_tasmax_monthly_max_trend.tasmax_monthly_max_trend.values[m,month,:,:] - era5_tasmax_monthly_max_trend.tasmax_monthly_max_trend.values[month,:,:]

    for month in range(12):
        print('month %d'%month)
        for xlat in range(cmip6_era5_tasmax_monthly_max_trend_bias_sig.shape[1]):
            for ylon in range(cmip6_era5_tasmax_monthly_max_trend_bias_sig.shape[2]):
                t, p = scipy.stats.ttest_1samp(cmip6_era5_tasmax_monthly_max_trend_bias[month,:, xlat, ylon], 0)
                cmip6_era5_tasmax_monthly_max_trend_bias_sig[month,xlat, ylon] = p


    # TASMAX GROWING SEASON MAX
    print('tasmax growing season max')
    cmip6_era5_tasmax_grow_max_trend_bias_sig = np.full(era5_tasmax_grow_max_trend.tasmax_grow_max_trend.shape, np.nan)
    cmip6_era5_tasmax_grow_max_trend_bias = np.full([cmip6_tasmax_grow_max_trend.model.size, era5_tasmax_grow_max_trend.tasmax_grow_max_trend.shape[0], era5_tasmax_grow_max_trend.tasmax_grow_max_trend.shape[1]], np.nan)

    for m, model in enumerate(cmip6_tasmax_grow_max_trend.model):
        cmip6_era5_tasmax_grow_max_trend_bias[m,:,:] = cmip6_tasmax_grow_max_trend.tasmax_grow_max_trend.values[m,:,:] - era5_tasmax_grow_max_trend.tasmax_grow_max_trend.values

    for xlat in range(cmip6_era5_tasmax_grow_max_trend_bias_sig.shape[0]):
        for ylon in range(cmip6_era5_tasmax_grow_max_trend_bias_sig.shape[1]):
            t, p = scipy.stats.ttest_1samp(cmip6_era5_tasmax_grow_max_trend_bias[:, xlat, ylon], 0)
            cmip6_era5_tasmax_grow_max_trend_bias_sig[xlat, ylon] = p

    # TASMAX GROWING SEASON MEAN
    print('tasmax growing season mean')
    cmip6_era5_tasmax_grow_mean_trend_bias_sig = np.full(era5_tasmax_grow_mean_trend.tasmax_grow_mean_trend.shape, np.nan)
    cmip6_era5_tasmax_grow_mean_trend_bias = np.full([cmip6_tasmax_grow_max_trend.model.size, era5_tasmax_grow_mean_trend.tasmax_grow_mean_trend.shape[0], era5_tasmax_grow_mean_trend.tasmax_grow_mean_trend.shape[1]], np.nan)

    for m, model in enumerate(cmip6_tasmax_grow_max_trend.model):
        cmip6_era5_tasmax_grow_mean_trend_bias[m,:,:] = cmip6_tasmax_grow_mean_trend.tasmax_grow_mean_trend.values[m,:,:] - era5_tasmax_grow_mean_trend.tasmax_grow_mean_trend.values

    for xlat in range(cmip6_era5_tasmax_grow_mean_trend_bias_sig.shape[0]):
        for ylon in range(cmip6_era5_tasmax_grow_mean_trend_bias_sig.shape[1]):
            t, p = scipy.stats.ttest_1samp(cmip6_era5_tasmax_grow_mean_trend_bias[:, xlat, ylon], 0)
            cmip6_era5_tasmax_grow_mean_trend_bias_sig[xlat, ylon] = p


    # EF GROWING SEASON MEAN
    print('ef growing season mean')
    cmip6_era5_ef_grow_trend_bias_sig = np.full(era5_ef_grow_trend.ef_grow_trend.shape, np.nan)
    cmip6_era5_ef_grow_trend_bias = np.full([cmip6_ef_grow_trend.model.size, era5_ef_grow_trend.ef_grow_trend.shape[0], era5_ef_grow_trend.ef_grow_trend.shape[1]], np.nan)

    for m, model in enumerate(cmip6_ef_grow_trend.model):
        cmip6_era5_ef_grow_trend_bias[m,:,:] = cmip6_ef_grow_trend.grow_ef.values[m,:,:] - era5_ef_grow_trend.ef_grow_trend.values

    for xlat in range(cmip6_era5_ef_grow_trend_bias_sig.shape[0]):
        for ylon in range(cmip6_era5_ef_grow_trend_bias_sig.shape[1]):
            t, p = scipy.stats.ttest_1samp(cmip6_era5_ef_grow_trend_bias[:, xlat, ylon], 0)
            cmip6_era5_ef_grow_trend_bias_sig[xlat, ylon] = p

            
    cmip6_era5_biases = {'cmip6_era5_tasmax_max_trend_bias':cmip6_era5_tasmax_max_trend_bias,
                            'cmip6_era5_tasmax_max_trend_bias_sig':cmip6_era5_tasmax_max_trend_bias_sig,
                        'cmip6_era5_tasmax_monthly_max_trend_bias':cmip6_era5_tasmax_monthly_max_trend_bias,
                        'cmip6_era5_tasmax_monthly_max_trend_bias_sig':cmip6_era5_tasmax_monthly_max_trend_bias_sig,
                         'cmip6_era5_tasmax_grow_max_trend_bias':cmip6_era5_tasmax_grow_max_trend_bias,
                         'cmip6_era5_tasmax_grow_max_trend_bias_sig':cmip6_era5_tasmax_grow_max_trend_bias_sig,
                         'cmip6_era5_tasmax_grow_mean_trend_bias':cmip6_era5_tasmax_grow_mean_trend_bias,
                         'cmip6_era5_tasmax_grow_mean_trend_bias_sig':cmip6_era5_tasmax_grow_mean_trend_bias_sig,
                         'cmip6_era5_ef_grow_trend_bias':cmip6_era5_ef_grow_trend_bias,
                         'cmip6_era5_ef_grow_trend_bias_sig':cmip6_era5_ef_grow_trend_bias_sig}
    
    with open('cmip6_output/bias/cmip6-era5-bias.dat', 'wb') as f:
        pickle.dump(cmip6_era5_biases, f)


tasmax annual max
tasmax monthly max
month 0
month 1
month 2
month 3
month 4
month 5
month 6
month 7
month 8
month 9
month 10
month 11
tasmax growing season max
tasmax growing season mean
ef growing season mean


In [219]:
# load pre-calculated bias by year

monthly_groups = cmip6_tasmax_monthly_max.groupby('time.month').groups

yearly_tasmax_max_bias = np.full([len(cmip6_models), era5_tasmax_max_regrid.time.values.shape[0], \
                                  era5_tasmax_max_regrid.lat.values.shape[0], \
                                  era5_tasmax_max_regrid.lon.values.shape[0]], np.nan)
yearly_tasmax_monthly_max_bias = np.full([len(cmip6_models), 12, len(monthly_groups[1]), \
                                  cmip6_tasmax_monthly_max.lat.values.shape[0], \
                                  cmip6_tasmax_monthly_max.lon.values.shape[0]], np.nan)
yearly_tasmax_mean_bias = np.full([len(cmip6_models), era5_tasmax_mean_regrid.time.values.shape[0], \
                                   era5_tasmax_mean_regrid.lat.values.shape[0], \
                                   era5_tasmax_mean_regrid.lon.values.shape[0]], np.nan)

yearly_tasmax_grow_max_bias = np.full([len(cmip6_models), era5_tasmax_grow_max_regrid.time.values.shape[0], \
                                  era5_tasmax_grow_max_regrid.lat.values.shape[0], \
                                  era5_tasmax_grow_max_regrid.lon.values.shape[0]], np.nan)
yearly_tasmax_grow_mean_bias = np.full([len(cmip6_models), era5_tasmax_grow_mean_regrid.time.values.shape[0], \
                                   era5_tasmax_grow_mean_regrid.lat.values.shape[0], \
                                   era5_tasmax_grow_mean_regrid.lon.values.shape[0]], np.nan)
yearly_ef_grow_bias = np.full([len(cmip6_models), era5_ef_grow_regrid.time.values.shape[0], \
                                   era5_ef_grow_regrid.lat.values.shape[0], \
                                   era5_ef_grow_regrid.lon.values.shape[0]], np.nan)

for m, model in enumerate(cmip6_models):
    print('loading yearly bias for %s'%model)
    with open('cmip6_output/bias/yearly-cmip6-era5-tasmax-max-bias-%s-%s.dat'%(region, model), 'rb') as f:
        yearly_tasmax_max_bias[m, :, :, :] = pickle.load(f)
    with open('cmip6_output/bias/yearly-cmip6-era5-tasmax-monthly-max-bias-%s-%s.dat'%(region, model), 'rb') as f:
        yearly_tasmax_monthly_max_bias[m, :, :, :, :] = pickle.load(f)
    with open('cmip6_output/bias/yearly-cmip6-era5-tasmax-mean-bias-%s-%s.dat'%(region, model), 'rb') as f:
        yearly_tasmax_mean_bias[m, :, :, :] = pickle.load(f)
        
    with open('cmip6_output/bias/yearly-cmip6-era5-tasmax-grow-max-bias-%s-%s.dat'%(region, model), 'rb') as f:
        yearly_tasmax_grow_max_bias[m, :, :, :] = pickle.load(f)
    with open('cmip6_output/bias/yearly-cmip6-era5-tasmax-grow-mean-bias-%s-%s.dat'%(region, model), 'rb') as f:
        yearly_tasmax_grow_mean_bias[m, :, :, :] = pickle.load(f)
    if os.path.isfile('cmip6_output/bias/yearly-cmip6-era5-ef-grow-bias-%s-%s.dat'%(region, model)):
        with open('cmip6_output/bias/yearly-cmip6-era5-ef-grow-bias-%s-%s.dat'%(region, model), 'rb') as f:
            yearly_ef_grow_bias[m, :, :, :] = pickle.load(f)


loading yearly bias for access-cm2
loading yearly bias for access-esm1-5
loading yearly bias for awi-cm-1-1-mr
loading yearly bias for bcc-csm2-mr
loading yearly bias for bcc-esm1
loading yearly bias for canesm5
loading yearly bias for ec-earth3
loading yearly bias for gfdl-cm4
loading yearly bias for gfdl-esm4
loading yearly bias for giss-e2-1-g
loading yearly bias for kace-1-0-g
loading yearly bias for fgoals-g3
loading yearly bias for inm-cm5-0
loading yearly bias for ipsl-cm6a-lr
loading yearly bias for miroc6
loading yearly bias for mpi-esm1-2-hr
loading yearly bias for mpi-esm1-2-lr
loading yearly bias for mri-esm2-0
loading yearly bias for noresm2-lm
loading yearly bias for noresm2-mm
loading yearly bias for sam0-unicon


In [234]:
# load deepak data

recalc = False

if not os.path.isfile('deepak_regions/deepak_maize_%s.nc'%region) or recalc:
    
    maize_yield_deepak_ds = xr.Dataset()
    soy_yield_deepak_ds = xr.Dataset()
    wheat_yield_deepak_ds = xr.Dataset()

    for y, year in enumerate(range(1979, 2013+1)):
        print('processing deepak for %d...'%year)
        cur_maize_yield_deepak_ds = xr.open_dataset('%s/Maize_yield_1970_2013/Maize_areaweightedyield_%d_ver12b.nc'%(dirDeepak, year))

        lonvals = cur_maize_yield_deepak_ds['longitude'].values
        lonvals[lonvals < 0] = 360+lonvals[lonvals < 0]
        cur_maize_yield_deepak_ds['longitude'] = lonvals

        latvals = cur_maize_yield_deepak_ds['latitude'].values
        cur_maize_yield_deepak_ds['latitude'] = latvals

        cur_maize_yield_deepak_ds = cur_maize_yield_deepak_ds.rename(latitude='lat', longitude='lon')

        cur_maize_yield_deepak_ds.load()

        regridder = xe.Regridder(cur_maize_yield_deepak_ds, regridMesh, 'bilinear')
        regridder.clean_weight_file()
        cur_maize_yield_deepak_ds_regrid = regridder(cur_maize_yield_deepak_ds.Data)

        tempDs_maize_yield = xr.Dataset()
        tempDs_maize_yield['maize_yield'] = xr.DataArray(data = np.squeeze(cur_maize_yield_deepak_ds_regrid.values, axis=1), 
                              dims   = ['time', 'lat', 'lon'],
                              coords = {'time':[datetime.datetime(year,1,1)], 'lat':regridMesh['lat'], 'lon':regridMesh['lon']},
                              attrs  = {'units'     : 't/ha'
                                })

        if y == 0:
            maize_yield_deepak_ds = tempDs_maize_yield
        else:
            maize_yield_deepak_ds = xr.concat([maize_yield_deepak_ds, tempDs_maize_yield], dim='time')
        
        
        
        cur_soy_yield_deepak_ds = xr.open_dataset('%s/Soy_yield_1970_2013/Soybean_areaweightedyield_%d_ver12b.nc'%(dirDeepak, year))

        lonvals = cur_soy_yield_deepak_ds['longitude'].values
        lonvals[lonvals < 0] = 360+lonvals[lonvals < 0]
        cur_soy_yield_deepak_ds['longitude'] = lonvals

        latvals = cur_soy_yield_deepak_ds['latitude'].values
        cur_soy_yield_deepak_ds['latitude'] = latvals

        cur_soy_yield_deepak_ds = cur_soy_yield_deepak_ds.rename(latitude='lat', longitude='lon')

        cur_soy_yield_deepak_ds.load()

        regridder = xe.Regridder(cur_soy_yield_deepak_ds, regridMesh, 'bilinear')
        regridder.clean_weight_file()
        cur_soy_yield_deepak_ds_regrid = regridder(cur_soy_yield_deepak_ds.Data)

        tempDs_soy_yield = xr.Dataset()
        tempDs_soy_yield['soy_yield'] = xr.DataArray(data = np.squeeze(cur_soy_yield_deepak_ds_regrid.values, axis=1), 
                              dims   = ['time', 'lat', 'lon'],
                              coords = {'time':[datetime.datetime(year,1,1)], 'lat':regridMesh['lat'], 'lon':regridMesh['lon']},
                              attrs  = {'units'     : 't/ha'
                                })

        if y == 0:
            soy_yield_deepak_ds = tempDs_soy_yield
        else:
            soy_yield_deepak_ds = xr.concat([soy_yield_deepak_ds, tempDs_soy_yield], dim='time')
        
        
        
        cur_wheat_yield_deepak_ds = xr.open_dataset('%s/Wheat_yield_1970_2013/Wheat_areaweightedyield_%d_ver12b.nc'%(dirDeepak, year))

        lonvals = cur_wheat_yield_deepak_ds['longitude'].values
        lonvals[lonvals < 0] = 360+lonvals[lonvals < 0]
        cur_wheat_yield_deepak_ds['longitude'] = lonvals

        latvals = cur_wheat_yield_deepak_ds['latitude'].values
        cur_wheat_yield_deepak_ds['latitude'] = latvals

        cur_wheat_yield_deepak_ds = cur_wheat_yield_deepak_ds.rename(latitude='lat', longitude='lon')

        cur_wheat_yield_deepak_ds.load()

        regridder = xe.Regridder(cur_wheat_yield_deepak_ds, regridMesh, 'bilinear')
        regridder.clean_weight_file()
        cur_wheat_yield_deepak_ds_regrid = regridder(cur_wheat_yield_deepak_ds.Data)

        tempDs_wheat_yield = xr.Dataset()
        tempDs_wheat_yield['wheat_yield'] = xr.DataArray(data = np.squeeze(cur_wheat_yield_deepak_ds_regrid.values, axis=1), 
                              dims   = ['time', 'lat', 'lon'],
                              coords = {'time':[datetime.datetime(year,1,1)], 'lat':regridMesh['lat'], 'lon':regridMesh['lon']},
                              attrs  = {'units'     : 't/ha'
                                })

        if y == 0:
            wheat_yield_deepak_ds = tempDs_wheat_yield
        else:
            wheat_yield_deepak_ds = xr.concat([wheat_yield_deepak_ds, tempDs_wheat_yield], dim='time')
            
    maize_yield_deepak_ds.to_netcdf('deepak_regions/deepak_maize_%s.nc'%region)
    soy_yield_deepak_ds.to_netcdf('deepak_regions/deepak_soy_%s.nc'%region)
    wheat_yield_deepak_ds.to_netcdf('deepak_regions/deepak_wheat_%s.nc'%region)
else:
    maize_yield_deepak_ds = xr.open_dataset('deepak_regions/deepak_maize_%s.nc'%region)
    soy_yield_deepak_ds = xr.open_dataset('deepak_regions/deepak_soy_%s.nc'%region)
    wheat_yield_deepak_ds = xr.open_dataset('deepak_regions/deepak_wheat_%s.nc'%region)